In [ ]:
import os
import random
import torch #pytorch
import librosa
import numpy as np
from datasets import load_dataset, Audio
from transformers import EncodecModel, AutoProcessor, GPT2Config, GPT2Model
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
# used to define neural network layers and setting up and training the network
from IPython.display import Audio as IPyAudio
import soundfile as sf
import csv
from typing import List
from tqdm import tqdm # for progress bars
import soundfile as sf
from audiomentations import Compose, AddGaussianNoise
import wandb

In [2]:
os.environ["WANDB_NOTEBOOK_NAME"] = "FoleyGen_Oct.ipynb"
wandb.login(key = '1a9688afc9e6c6bf3585eecf13438d302bdbcd73')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ssr9055 (dl4m_final). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ssr9055/.netrc


True

In [3]:
#Resuming from last run
wandb.init(
    project="FoleyGen_Oct",
    id="7ue2sn90",  
    resume="must"  
)

## Splitting datasets

In [4]:
clas_dict = {
    "DogBark": 0,
    "Footstep": 1,
    "Gunshot": 2,
    "Keyboard": 3,
    "MovingMotorVehicle": 4,
    "Rain": 5,
    "SneezeCough": 6,
}

In [1]:
# Update with your path
dataset_path = ""
output_csv_path = ""

In [6]:
def load_dataset_from_csv(csv_path: str):
    training_files = []
    valid_files = []
    test_files = []

    with open(csv_path, "r") as csv_file:
        reader = csv.DictReader(csv_file)
        for row in reader:
            file_info = {
                "file_path": row["filepath"],
                "class_id": clas_dict.get(os.path.basename(os.path.dirname(row["filepath"])), None),
            }
            if row["split"] == "train":
                training_files.append(file_info)
            elif row["split"] == "validation":
                valid_files.append(file_info)
            elif row["split"] == "test":
                test_files.append(file_info)

    return training_files, valid_files, test_files



def split_dataset_files(dataset_path: str, csv_path: str):
   
    # Check if the CSV file exists
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"The specified CSV file does not exist: {csv_path}")
    
    # Load splits from the CSV
    print(f"Loading splits from existing CSV: {csv_path}")
    return load_dataset_from_csv(csv_path)



In [7]:
# Split the dataset
training_files, valid_files, test_files = split_dataset_files(dataset_path, csv_path=output_csv_path)

Loading splits from existing CSV: /scratch/ssr9055/my_env/dataset_splits.csv


## Initalizing Encodec Model


In [10]:
# Initialize EnCodec model
from encodec import EncodecModel

encodec_model = EncodecModel.encodec_model_24khz()
encodec_model.set_target_bandwidth(6.0)

# Define codebook_size and num_quantizers based on the actual model
codebook_size = 1024  # Since max index is up to 1023
num_quantizers = 8     # For EnCodec model with target bandwidth of 6.0

/ext3/miniforge3/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


# Gaussian Noise Augmentation and converting raw audio to tokens with Encodec

In [11]:


# Define augmentation
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
])

def encode_audio(file_path, model, sr=24000, apply_augmentation = False):
    import soundfile as sf
    audio, original_sr = sf.read(file_path)

    if len(audio) == 0:
        audio = np.zeros(1)

    if apply_augmentation:
        # Apply augmentation (CPU)
        audio = augment(samples=audio, sample_rate=sr)

    

    # Convert to tensor and move to GPU
    audio_tensor = torch.from_numpy(audio).float().unsqueeze(0).unsqueeze(0)
    audio_tensor = audio_tensor.to(next(model.parameters()).device)

    with torch.no_grad():
        # Encode the audio using EnCodec (GPU)
        encoded_frames = model.encode(audio_tensor)

    codes_list = [frame[0] for frame in encoded_frames]  
    codes = torch.cat(codes_list, dim=2)

    codes = codes.squeeze(0).permute(1, 0).long()

    num_quantizers = codes.shape[1]

    for q in range(num_quantizers):
        max_index = codes[:, q].max().item()
        min_index = codes[:, q].min().item()

    return codes


## Functions for applying and removing delay pattern

In [12]:
def apply_delay_pattern(codes):
    
    num_frames, num_quantizers = codes.shape
    
    max_delay = num_quantizers - 1
    
    padding_value = codebook_size  
    
    delayed_codes = torch.full((num_frames + max_delay, num_quantizers), fill_value=padding_value, dtype=codes.dtype)
    
    for q in range(num_quantizers):
        delayed_codes[q:q + num_frames, q] = codes[:, q]
        
    return delayed_codes  # Shape: [num_frames + max_delay, num_quantizers]

def remove_delay_pattern(delayed_codes, num_quantizers):
    
    num_frames = delayed_codes.shape[0] - (num_quantizers - 1)
    
    codes = torch.zeros(num_frames, num_quantizers, dtype=delayed_codes.dtype)
    
    for q in range(num_quantizers):
        codes[:, q] = delayed_codes[q:q + num_frames, q]
        
    return codes  # Shape: [num_frames, num_quantizers]

## Create AudioDataset Class and Data Loaders

In [13]:
class AudioDataset(Dataset):
    def __init__(self, file_list, encodec_model, max_length=300, codebook_size=1024, apply_augmentation = False):
        self.file_list = file_list
        self.encodec_model = encodec_model
        self.max_length = max_length
        self.codebook_size = codebook_size
        self.vocab_size = self.codebook_size + 1  # +1 for padding
        self.num_quantizers = 8
        self.apply_augmentation = apply_augmentation

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_info = self.file_list[idx]
        file_path = file_info["file_path"]
        class_id = file_info["class_id"]  # Get the class ID
        codes = encode_audio(file_path, self.encodec_model, apply_augmentation=self.apply_augmentation)
        delayed_codes = apply_delay_pattern(codes)

        # Truncate or pad sequences to max_length
        input_ids = delayed_codes
        padding_value = self.vocab_size - 1
        if input_ids.shape[0] > self.max_length:
            input_ids = input_ids[:self.max_length, :]
        else:
            pad_length = self.max_length - input_ids.shape[0]
            padding = torch.full((pad_length, input_ids.shape[1]), padding_value, dtype=torch.long)
            input_ids = torch.cat([input_ids, padding], dim=0)

        return input_ids, class_id  # Return the class ID along with the input_ids




# Create Data Loaders
max_sequence_length = 300
batch_size = 4

train_dataset = AudioDataset(training_files, encodec_model, max_length=max_sequence_length, apply_augmentation = True)
valid_dataset = AudioDataset(valid_files, encodec_model, max_length=max_sequence_length, apply_augmentation = False)
test_dataset = AudioDataset(test_files, encodec_model, max_length=max_sequence_length, apply_augmentation = False)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

## Define AudioGPT2 Model


In [14]:
class AudioGPT2(nn.Module):
    def __init__(self, num_quantizers=8, codebook_size=1024, hidden_size=768, num_classes=7):
        super(AudioGPT2, self).__init__()
        self.num_quantizers = num_quantizers
        self.codebook_size = codebook_size
        self.vocab_size = self.codebook_size + 1  # +1 for padding
        self.total_vocab_size = self.num_quantizers * self.vocab_size

        # Custom embedding layer for the audio tokens
        self.embedding = nn.Embedding(self.total_vocab_size, hidden_size)

        # Embedding layer for class labels
        self.class_embedding = nn.Embedding(num_classes, hidden_size)

        # Loading pre-trained GPT-2 model
        self.gpt2 = GPT2Model.from_pretrained('gpt2')

        # Output layer
        self.output_layer = nn.Linear(hidden_size, self.total_vocab_size)

    def forward(self, input_ids=None, class_id=None):
        # input_ids: [batch_size, seq_length, num_quantizers]
        # class_id: [batch_size]

        # Offset tokens for each quantizer to avoid overlap
        offset = torch.arange(self.num_quantizers, device=input_ids.device) * self.vocab_size
        offset = offset.unsqueeze(0).unsqueeze(0)
        input_ids = input_ids + offset  

        # Get audio embeddings
        embeddings = self.embedding(input_ids).sum(dim=2)  # Sum across quantizers

        # Get class embeddings and expand to match the sequence length
        class_embeddings = self.class_embedding(class_id).unsqueeze(1).expand(-1, embeddings.size(1), -1)

        # Concatenate class embeddings with audio embeddings
        combined_embeddings = embeddings + class_embeddings

        # Pass through GPT-2
        gpt_outputs = self.gpt2(inputs_embeds=combined_embeddings)

        # Output layer
        logits = self.output_layer(gpt_outputs.last_hidden_state)
        return logits

## Train model

In [15]:
# Initialize the model, loss function, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AudioGPT2(num_quantizers=num_quantizers, codebook_size=codebook_size).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=model.vocab_size - 1)  # Ignore padding token
optimizer = optim.Adam(model.parameters(), lr=5e-5, weight_decay = 1e-4)

#### For starting a new training run

In [18]:

num_epochs = 140

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
        input_ids, class_id = batch
        input_ids, class_id = input_ids.to(device), class_id.to(device)
        
        optimizer.zero_grad()
        
        

        # Prepare inputs and targets
        inputs = input_ids[:, :-1, :]  # [batch_size, seq_length-1, num_quantizers]
        targets = input_ids[:, 1:, :]  # [batch_size, seq_length-1, num_quantizers]

        # Forward pass
        logits = model(input_ids = inputs, class_id = class_id)  # [batch_size, seq_length-1, total_vocab_size]

        # Reshape logits and targets
        batch_size, seq_length_minus1, _ = inputs.shape
        logits = logits.reshape(batch_size * seq_length_minus1, model.total_vocab_size)
        targets = targets.reshape(batch_size * seq_length_minus1, model.num_quantizers)

        # Compute loss per quantizer
        loss = 0
        for q in range(model.num_quantizers):
            q_targets = targets[:, q]  # [batch_size * seq_length_minus1]
            q_offset = q * model.vocab_size
            q_logits = logits[:, q_offset : q_offset + model.vocab_size]  # [batch_size * seq_length_minus1, vocab_size]
            loss += criterion(q_logits, q_targets)
        loss = loss / model.num_quantizers  # Average over quantizers
        

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_loss:.4f}")
    wandb.log({"Training Loss": avg_loss, "Epoch": epoch + 1})

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch+1}"):
            input_ids, class_id = batch
            input_ids, class_id = input_ids.to(device), class_id.to(device)
            inputs = input_ids[:, :-1, :]
            targets = input_ids[:, 1:, :]
            logits = model(input_ids=inputs, class_id=class_id)
            batch_size, seq_length_minus1, _ = inputs.shape
            logits = logits.reshape(batch_size * seq_length_minus1, model.total_vocab_size)
            targets = targets.reshape(batch_size * seq_length_minus1, model.num_quantizers)

            loss = 0
            for q in range(model.num_quantizers):
                q_targets = targets[:, q]
                q_offset = q * model.vocab_size
                q_logits = logits[:, q_offset : q_offset + model.vocab_size]
                loss += criterion(q_logits, q_targets)
            loss = loss / model.num_quantizers
            val_loss += loss.item()
    avg_val_loss = val_loss / len(valid_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}")
    wandb.log({"Validation Loss": avg_val_loss, "Epoch": epoch + 1})

    if (epoch + 1) % 10 == 0:
        checkpoint_path = f"augmodel_checkpoint_l2_{epoch + 1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved for epoch {epoch + 1} at {checkpoint_path}")

wandb.finish()


Training Epoch 1:   0%|          | 0/1102 [00:00<?, ?it/s]/ext3/miniforge3/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
Training Epoch 1: 100%|██████████| 1102/1102 [14:54<00:00,  1.23it/s]


Epoch 1/140, Training Loss: 4.9914


Validation Epoch 1: 100%|██████████| 138/138 [01:42<00:00,  1.34it/s]


Epoch 1/140, Validation Loss: 4.4233


Training Epoch 2: 100%|██████████| 1102/1102 [13:49<00:00,  1.33it/s]


...

Epoch 28/140, Training Loss: 4.3198


Validation Epoch 28: 100%|██████████| 138/138 [01:32<00:00,  1.50it/s]


Epoch 28/140, Validation Loss: 4.0326


Training Epoch 29:  50%|█████     | 555/1102 [06:18<06:12,  1.47it/s]


KeyboardInterrupt: 

In [16]:
# Resuming previous run
checkpoint_path = "/scratch/ssr9055/my_env/BEST_CHECKPOINT_AUG_L2.pth"


start_epoch = 0
best_val_loss = float('inf') 

if os.path.exists(checkpoint_path):
    print(f"Loading checkpoint from {checkpoint_path}")
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1  # Resume from the next epoch
    print(f"Resuming training from epoch {start_epoch}")


#Number of additional epochs to train
num_epochs = 200

# Adjust the training loop to account for start_epoch
for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch}"):
        
        input_ids, class_id = batch  # Get input IDs and class IDs
        input_ids, class_id = input_ids.to(device), class_id.to(device)

        optimizer.zero_grad()

        inputs = input_ids[:, :-1, :]  # Input tokens
        targets = input_ids[:, 1:, :]  # Target tokens

        # Forward pass (use keyword arguments)
        logits = model(input_ids=inputs, class_id=class_id)  # Pass class_id using keyword args

        # Reshape logits and targets
        batch_size, seq_length_minus1, _ = inputs.shape
        logits = logits.reshape(batch_size * seq_length_minus1, model.total_vocab_size)
        targets = targets.reshape(batch_size * seq_length_minus1, model.num_quantizers)

        # Compute loss per quantizer
        loss = 0
        for q in range(model.num_quantizers):
            q_targets = targets[:, q]  # [batch_size * seq_length_minus1]
            q_offset = q * model.vocab_size
            q_logits = logits[:, q_offset : q_offset + model.vocab_size]  # [batch_size * seq_length_minus1, vocab_size]
            loss += criterion(q_logits, q_targets)
        loss = loss / model.num_quantizers  # Average over quantizers

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch}/{start_epoch + num_epochs}, Training Loss: {avg_loss:.4f}")
    wandb.log({"Training Loss": avg_loss, "Epoch": epoch + 1})


    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader, desc=f"Validation Epoch {epoch}"):
            input_ids, class_id = batch  # Ensure you extract both input_ids and class_id
            input_ids, class_id = input_ids.to(device), class_id.to(device)

            inputs = input_ids[:, :-1, :]
            targets = input_ids[:, 1:, :]

            # Forward pass (use keyword arguments)
            logits = model(input_ids=inputs, class_id=class_id)  # Pass class_id using keyword args

            batch_size, seq_length_minus1, _ = inputs.shape
            logits = logits.reshape(batch_size * seq_length_minus1, model.total_vocab_size)
            targets = targets.reshape(batch_size * seq_length_minus1, model.num_quantizers)

            loss = 0
            for q in range(model.num_quantizers):
                q_targets = targets[:, q]
                q_offset = q * model.vocab_size
                q_logits = logits[:, q_offset : q_offset + model.vocab_size]
                loss += criterion(q_logits, q_targets)
            loss = loss / model.num_quantizers
            val_loss += loss.item()

    avg_val_loss = val_loss / len(valid_loader)
    print(f"Epoch {epoch}/{start_epoch + num_epochs}, Validation Loss: {avg_val_loss:.4f}")
    wandb.log({"Validation Loss": avg_val_loss, "Epoch": epoch + 1})

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        checkpoint_path = f"BEST_CHECKPOINT_AUG_L2.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
            'best_val_loss': best_val_loss
        }, checkpoint_path)
        print(f"New best validation loss: {best_val_loss:.4f}. Checkpoint saved at {checkpoint_path}")


    if (epoch + 1) % 10 == 0:
        checkpoint_path = f"model_checkpoint_epoch_{epoch + 1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': avg_loss,
        }, checkpoint_path)
        print(f"Checkpoint saved for epoch {epoch + 1} at {checkpoint_path}")

wandb.finish()

Loading checkpoint from /scratch/ssr9055/my_env/BEST_CHECKPOINT_AUG_L2.pth


/state/partition1/job-53814134/ipykernel_2298929/760857916.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)


Resuming training from epoch 280


Training Epoch 280:   0%|          | 0/1102 [00:00<?, ?it/s]/ext3/miniforge3/lib/python3.12/site-packages/audiomentations/core/transforms_interface.py:62: UserWarning: Warning: input samples dtype is np.float64. Converting to np.float32
  warnings.warn(
Training Epoch 280: 100%|██████████| 1102/1102 [12:03<00:00,  1.52it/s]


Epoch 280/480, Training Loss: 4.0173


...

Epoch 300/480, Training Loss: 4.0263


Validation Epoch 300: 100%|██████████| 138/138 [01:04<00:00,  2.13it/s]


Epoch 300/480, Validation Loss: 3.7663
New best validation loss: 3.7663. Checkpoint saved at BEST_CHECKPOINT_AUG_L2.pth


Training Epoch 301:  49%|████▉     | 538/1102 [04:52<05:07,  1.84it/s]


KeyboardInterrupt: 

### Generating audio

In [20]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
import soundfile as sf
from IPython.display import Audio as IPyAudio

checkpoint_path = "/scratch/ssr9055/my_env/BEST_CHECKPOINT_AUG_L2.pth"

# Helper function to remove delay pattern
def remove_delay_pattern(delayed_codes, num_quantizers):
    num_frames = delayed_codes.shape[0] - (num_quantizers - 1)
    codes = torch.zeros(num_frames, num_quantizers, dtype=delayed_codes.dtype)
    for q in range(num_quantizers):
        codes[:, q] = delayed_codes[q:q + num_frames, q]
    return codes  # Shape: [num_frames, num_quantizers]

# Function to generate audio
def generate_audio(model, encodec_model, class_id, num_quantizers=8, codebook_size=1024, max_length=600, temperature=1.0, device='cpu'):
    model.eval()  # Set model to evaluation mode
    encodec_model.eval()  # Set encodec_model to evaluation mode

    model.to(device)  # Move model to the specified device

    # Start with padding tokens for each quantizer
    start_token = codebook_size  # Padding token index
    input_ids = torch.full((1, 1, num_quantizers), start_token, dtype=torch.long, device=device)  # Shape: [1, 1, num_quantizers]

    # Convert the class_id to a tensor and move to device
    class_id_tensor = torch.tensor([class_id], device=device)

    generated = []  # List to hold generated tokens

    with torch.no_grad():  # Disable gradient calculations for generation
        for _ in tqdm(range(max_length), desc="Generating Audio"):
            # Forward pass through the model, passing class_id_tensor
            logits = model(input_ids=input_ids, class_id=class_id_tensor)  # [1, seq_length, total_vocab_size]
            logits = logits[:, -1, :]  # [1, total_vocab_size] - Get the logits for the last time step

            # Apply temperature to control randomness
            logits = logits / temperature

            next_tokens = []  # List to store the next token for each quantizer

            # Sample next token for each quantizer
            for q in range(num_quantizers):
                q_offset = q * (codebook_size + 1)
                q_logits = logits[:, q_offset:q_offset + codebook_size + 1]  # [1, vocab_size]
                q_probs = F.softmax(q_logits, dim=-1)  # Convert logits to probabilities
                q_next_token = torch.multinomial(q_probs, num_samples=1)  # Sample next token
                q_next_token = q_next_token.squeeze(1)  # Remove extra dimension

                # If the sampled token is the padding token, replace it with a valid token
                q_next_token_value = q_next_token.item()
                if q_next_token_value == codebook_size:
                    q_next_token_value = torch.randint(0, codebook_size, (1,)).item()

                next_tokens.append(torch.tensor([q_next_token_value], device=device, dtype=torch.long))

            # Stack the next tokens for each quantizer and append to generated sequence
            next_tokens = torch.stack(next_tokens, dim=1)  # Shape: [1, num_quantizers]
            generated.append(next_tokens.squeeze(0))  # Append generated tokens
            input_ids = torch.cat([input_ids, next_tokens.unsqueeze(0)], dim=1)  # Update input_ids for the next time step

    # Stack the generated tokens to form the final token sequence
    generated_tokens = torch.stack(generated, dim=0)  # [seq_length, num_quantizers]

    # Remove the delay pattern from generated tokens
    codes = remove_delay_pattern(generated_tokens, num_quantizers)

    # Add batch dimension to match the input shape for decode
    codes = codes.unsqueeze(0)

    # Ensure codes are within valid codebook size
    codes = codes.clamp(0, codebook_size - 1)

    # Decode the audio using the EnCodec model
    try:
        with torch.no_grad():
            codes = codes.permute(0, 2, 1)  # [batch_size, num_quantizers, num_frames]
            encoded_frames = [(codes.to(next(encodec_model.parameters()).device), None)]
            decoded_audio = encodec_model.decode(encoded_frames)
            audio = decoded_audio.squeeze().cpu().detach().numpy()  # Convert to numpy array
        return audio
    except Exception as e:
        print(f"Error during decoding: {str(e)}")
        return None


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
max_length = 350  # Adjust as needed
temperature = 0.7  # Adjust as needed

checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# Loop over each category in clas_dict
for class_name, class_id in clas_dict.items():
    print(f"Generating audio for category: {class_name}")

    generated_audio = generate_audio(
        model,
        encodec_model,
        class_id=class_id,  # Pass the class_id for each category
        num_quantizers=num_quantizers,
        codebook_size=codebook_size,
        max_length=max_length,
        temperature=temperature,
        device=device
    )

    if generated_audio is not None:
        # Save the generated audio to a file (optional)
        output_filename = f'generated_audio_{class_name}.wav'
        sf.write(output_filename, generated_audio, 24000)

        # Display the audio
        display(IPyAudio(output_filename))
    else:
        print(f"Audio generation failed for category: {class_name}")


/state/partition1/job-53814134/ipykernel_2298929/2022961031.py:93: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=device

Generating audio for category: DogBark


Generating Audio: 100%|██████████| 350/350 [00:02<00:00, 131.70it/s]


Generating audio for category: Footstep


Generating Audio: 100%|██████████| 350/350 [00:02<00:00, 131.57it/s]


...

Generating Audio: 100%|██████████| 350/350 [00:02<00:00, 132.87it/s]


Generating audio for category: Rain


Generating Audio: 100%|██████████| 350/350 [00:02<00:00, 132.87it/s]


Generating audio for category: SneezeCough


Generating Audio: 100%|██████████| 350/350 [00:02<00:00, 131.23it/s]
